In [1]:
import numpy as np 
import pandas as pd 
import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from scipy import sparse
import warnings
import time
import sys
import os
import re
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',None)
pd.set_option('max_colwidth',100)

In [2]:
train = pd.read_csv('jinnan_round1_train_20181227.csv', encoding = 'gb18030')
test  = pd.read_csv('jinnan_round1_testA_20181227.csv', encoding = 'gb18030')

In [3]:
for df in [train, test]:
    df.drop(['B3', 'B13', 'A13', 'A18', 'A23'], axis=1, inplace=True)

In [4]:
good_cols = list(train.columns)
for col in train.columns:
    rate = train[col].value_counts(normalize=True, dropna=False).values[0]
    if rate > 0.9:
        good_cols.remove(col)
        print(col,rate)

train = train[train['收率']>0.87]
        
train = train[good_cols]
good_cols.remove('收率')
test  = test[good_cols]

A1 0.9863896848137536
A2 0.9699140401146131
A3 0.9570200573065902
A4 0.9570200573065902
B2 0.9842406876790831


In [5]:
target = train['收率']
del train['收率']
data = pd.concat([train,test],axis=0,ignore_index=True)
data = data.fillna(-1)

In [6]:
def timeTranSecond(t):
    try:
        t,m,s=t.split(":")
    except:
        if t=='1900/1/9 7:00':
            return 7*3600/3600
        elif t=='1900/1/1 2:30':
            return (2*3600+30*60)/3600
        elif t==-1:
            return -1
        else:
            return 0
    
    try:
        tm = (int(t)*3600+int(m)*60+int(s))/3600
    except:
        return (30*60)/3600
    
    return tm
for f in ['A5','A7','A9','A11','A14','A16','A24','A26','B5','B7']:
    data[f] = data[f].apply(timeTranSecond)

def getDuration(se):
    try:
        sh,sm,eh,em=re.findall(r"\d+\.?\d*",se)
    except:
        if se == -1:
            return -1 
        
    try:
        if int(sh)>int(eh):
            tm = (int(eh)*3600+int(em)*60-int(sm)*60-int(sh)*3600)/3600 + 24
        else:
            tm = (int(eh)*3600+int(em)*60-int(sm)*60-int(sh)*3600)/3600
    except:
        if se=='19:-20:05':
            return 1
        elif se=='15:00-1600':
            return 1
    
    return tm

def get_start(se):
    try:
        sh, sm, eh, em = re.findall(r'\d+\.?d*', se)
    except:
        if se == -1:
            return -1
    try:
        tm = (int(eh) * 3600 + int(em) * 60)/3600
    except:
        if se == '19:-20:05':
            return 19
        elif se == '15:00-1600':
            return 15
    return tm
for f in ['A20','A28','B4','B9','B10','B11']:
    data[f+'_diff'] = data.apply(lambda df: getDuration(df[f]), axis=1)
for f in ['A20','A28','B4','B9','B10','B11']:
    data[f] = data.apply(lambda df: get_start(df[f]), axis=1)

In [7]:
data['样本id'] = data['样本id'].apply(lambda x: int(x.split('_')[1]))

categorical_columns = [f for f in data.columns if f not in ['样本id']]
numerical_columns = [f for f in data.columns if f not in categorical_columns]

In [8]:
for f in categorical_columns:
    data[f] = data[f].map(dict(zip(data[f].unique(), range(0, data[f].nunique()))))
train = data[:train.shape[0]]
test  = data[train.shape[0]:]
print(train.shape)
print(test.shape)

(1381, 39)
(150, 39)


In [9]:
train['target'] = target
train['intTarget'] = pd.cut(train['target'], 5, labels=False)
train = pd.get_dummies(train, columns=['intTarget'])
li = ['intTarget_0.0','intTarget_1.0','intTarget_2.0','intTarget_3.0','intTarget_4.0']
mean_columns = []
for f1 in categorical_columns:
    cate_rate = train[f1].value_counts(normalize=True, dropna=False).values[0]
    if cate_rate < 0.90:
        for f2 in li:
            col_mean = f1 + f2 + '_mean'
            col_name = 'B14_to_'+f1+"_"+f2+'_mean'
            mean_columns.append(col_name)
            #mean_columns.append(col_mean)
            order_label = train.groupby([f1])[f2].mean()
            train[col_name] = train['B14'].map(order_label)
            train[col_mean] = train[f1].map(order_label)
            miss_rate = train[col_name].isnull().sum() * 100 / train[col_name].shape[0]
            if miss_rate > 0:
                train = train.drop([col_name], axis=1)
                mean_columns.remove(col_name)
            else:
                test[col_name] = test['B14'].map(order_label)
            miss_rate = train[col_mean].isnull().sum() * 100 / train[col_mean].shape[0]
            if miss_rate > 0:
                train = train.drop([col_mean], axis=1)
                mean_columns.remove(col_mean)
            else:
                test[col_mean] = test[f1].map(order_label)
            
                
train.drop(li, axis=1, inplace=True)
print(train.shape)
print(test.shape)

(1381, 335)
(150, 334)


In [10]:
corr_col = mean_columns + ['target']
corr_spy = train
corr_spy = pd.DataFrame(corr_spy, columns=corr_col)
good_mean = list(corr_spy.corr()['target'].abs().sort_values()[int(len(mean_columns)*0.50):-1].index)

In [ ]:
# for f1 in good_mean:
#     for f2 in good_mean:
#         col_name = f1 + 'X' + f2
#         if f1 == f2:
#             pass
#         else:
#             good_mean.append(col_name)
#             train[col_name] = train[f1] * train[f2]
#             test[col_name] = test[f1] * test[f2]

In [11]:
X_train = train[mean_columns+numerical_columns].values
X_test = test[mean_columns+numerical_columns].values
# one hot
enc = OneHotEncoder()
for f in categorical_columns:
    enc.fit(data[f].values.reshape(-1, 1))
    X_train = sparse.hstack((X_train, enc.transform(train[f].values.reshape(-1, 1))), 'csr')
    X_test = sparse.hstack((X_test, enc.transform(test[f].values.reshape(-1, 1))), 'csr')
print(X_train.shape)
print(X_test.shape)

(1381, 1686)
(150, 1686)


In [12]:
y_train = target.values

In [13]:
param = {'num_leaves': 300,
         'min_data_in_leaf': 12, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "boosting": "gbdt",
         "feature_fraction": 0.8,
         "bagging_freq": 1,
         "bagging_fraction": 0.8,
         "bagging_seed": 6,
         "metric": 'mse',
         "lambda_l1": 0.1,
         "verbosity": -1}
folds = KFold(n_splits=5, shuffle=True, random_state=666)
oof_lgb = np.zeros(len(train))
predictions_lgb = np.zeros(len(test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_+1))
    trn_data = lgb.Dataset(X_train[trn_idx], y_train[trn_idx])
    val_data = lgb.Dataset(X_train[val_idx], y_train[val_idx])

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=200, early_stopping_rounds = 100)
    oof_lgb[val_idx] = clf.predict(X_train[val_idx], num_iteration=clf.best_iteration)
    
    predictions_lgb += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.8f}".format(mean_squared_error(oof_lgb, target)))

fold n°1
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 0.000172086	valid_1's l2: 0.000217744
[400]	training's l2: 0.000115985	valid_1's l2: 0.000158768
[600]	training's l2: 9.88312e-05	valid_1's l2: 0.000143687
[800]	training's l2: 9.14779e-05	valid_1's l2: 0.000138355
[1000]	training's l2: 8.70531e-05	valid_1's l2: 0.000135142
[1200]	training's l2: 8.37233e-05	valid_1's l2: 0.000133138
[1400]	training's l2: 8.11962e-05	valid_1's l2: 0.000131632
[1600]	training's l2: 7.90949e-05	valid_1's l2: 0.000130356
[1800]	training's l2: 7.74613e-05	valid_1's l2: 0.000129425
[2000]	training's l2: 7.59843e-05	valid_1's l2: 0.000128693
[2200]	training's l2: 7.46075e-05	valid_1's l2: 0.000128212
Early stopping, best iteration is:
[2209]	training's l2: 7.45401e-05	valid_1's l2: 0.000128198
fold n°2
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 0.000176084	valid_1's l2: 0.000190293
[400]	training's l2: 0.000121218	valid_1's l2:

In [14]:
xgb_params = {'eta': 0.003, 'max_depth': 12, 'subsample': 0.8, 'colsample_bytree': 0.8, 
          'objective': 'reg:linear', 'eval_metric': 'rmse', 'silent': True, 'nthread': 4}

folds = KFold(n_splits=5, shuffle=True, random_state=666)
oof_xgb = np.zeros(len(train))
predictions_xgb = np.zeros(len(test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_+1))
    trn_data = xgb.DMatrix(X_train[trn_idx], y_train[trn_idx])
    val_data = xgb.DMatrix(X_train[val_idx], y_train[val_idx])

    watchlist = [(trn_data, 'train'), (val_data, 'valid_data')]
    clf = xgb.train(dtrain=trn_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200, verbose_eval=100, params=xgb_params)
    oof_xgb[val_idx] = clf.predict(xgb.DMatrix(X_train[val_idx]), ntree_limit=clf.best_ntree_limit)
    predictions_xgb += clf.predict(xgb.DMatrix(X_test), ntree_limit=clf.best_ntree_limit) / folds.n_splits
    
print("CV score: {:<8.8f}".format(mean_squared_error(oof_xgb, target)))

fold n°1
[0]	train-rmse:0.424062	valid_data-rmse:0.423535
Multiple eval metrics have been passed: 'valid_data-rmse' will be used for early stopping.

Will train until valid_data-rmse hasn't improved in 200 rounds.
[100]	train-rmse:0.314519	valid_data-rmse:0.313953
[200]	train-rmse:0.233443	valid_data-rmse:0.232877
[300]	train-rmse:0.173455	valid_data-rmse:0.17296
[400]	train-rmse:0.129094	valid_data-rmse:0.128722
[500]	train-rmse:0.09629	valid_data-rmse:0.096041
[600]	train-rmse:0.072074	valid_data-rmse:0.071964
[700]	train-rmse:0.054209	valid_data-rmse:0.054299
[800]	train-rmse:0.041002	valid_data-rmse:0.04136
[900]	train-rmse:0.031284	valid_data-rmse:0.031998
[1000]	train-rmse:0.024127	valid_data-rmse:0.02526
[1100]	train-rmse:0.018893	valid_data-rmse:0.020482
[1200]	train-rmse:0.015047	valid_data-rmse:0.017178
[1300]	train-rmse:0.012253	valid_data-rmse:0.014976
[1400]	train-rmse:0.01024	valid_data-rmse:0.013534
[1500]	train-rmse:0.008761	valid_data-rmse:0.012604
[1600]	train-rmse:0.

In [15]:
train_stack = np.vstack([oof_lgb,oof_xgb]).transpose()
test_stack = np.vstack([predictions_lgb, predictions_xgb]).transpose()

folds_stack = RepeatedKFold(n_splits=5, n_repeats=2, random_state=4590)
oof_stack = np.zeros(train_stack.shape[0])
predictions = np.zeros(test_stack.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds_stack.split(train_stack,target)):
    print("fold {}".format(fold_))
    trn_data, trn_y = train_stack[trn_idx], target.iloc[trn_idx].values
    val_data, val_y = train_stack[val_idx], target.iloc[val_idx].values
    
    clf_3 = BayesianRidge()
    clf_3.fit(trn_data, trn_y)
    
    oof_stack[val_idx] = clf_3.predict(val_data)
    predictions += clf_3.predict(test_stack) / 10
    
mean_squared_error(target.values, oof_stack)

fold 0
fold 1
fold 2
fold 3
fold 4
fold 5
fold 6
fold 7
fold 8
fold 9


0.00012074397189393701

In [17]:
sub_df = pd.read_csv('jinnan_round1_submit_20181227.csv', header=None)
sub_df[1] = predictions
sub_df[1] = sub_df[1].apply(lambda x:round(x, 3))
sub_df.to_csv("submit.csv", index=False, header=None)